In [24]:
!pip install optuna
!pip install efficientnet_pytorch


In [25]:

import torch
import pytorch_lightning as pl
import optuna
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18, resnet50
from efficientnet_pytorch import EfficientNet
import wandb

In [26]:
# Initialize WandB
wandb.init()

In [27]:
class ImageClassificationModel(pl.LightningModule):
    def __init__(self, backbone, learning_rate):
        super().__init__()
        self.backbone = backbone
        self.learning_rate = learning_rate
        
        # Define backbone architecture
        if backbone == 'ResNet':
            self.model = resnet18(pretrained=True)
            num_ftrs = self.model.fc.in_features
            self.model.fc = torch.nn.Linear(num_ftrs, 6)
        elif backbone == 'EfficientNet':
            self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=6)
            
        self.criterion = torch.nn.CrossEntropyLoss()    
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.model(x)
        loss = self.criterion(logits, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.model(x)
        loss = self.criterion(logits, y)
        self.log('val_loss', loss)
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.model(x)
        loss = self.criterion(logits, y)
        self.log('test_loss', loss)
        

In [28]:
# Load data
train_data = ImageFolder(root='archive/seg_train/seg_train/', transform=transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
]))
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

test_data = ImageFolder(root="archive/seg_test/seg_test/", transform=transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
]))
test_loader = DataLoader(test_data, batch_size=64)



In [29]:
# Define objective function for Optuna
def objective(trial):
    backbone = trial.suggest_categorical('backbone', ['ResNet', 'EfficientNet'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    model = ImageClassificationModel(backbone=backbone, learning_rate=learning_rate)
    trainer = pl.Trainer(max_epochs=1)  # Change max_epochs to 1
    trainer.fit(model, train_loader, test_loader)
    
    # return validation loss as the metric to minimize
    return trainer.logged_metrics['val_loss'].item()

In [30]:
# Optuna hyperparameter optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1)

# Get best hyperparameters
best_params = study.best_params
best_backbone = best_params['backbone']
best_learning_rate = best_params['learning_rate']

# Train model with best hyperparameters
best_model = ImageClassificationModel(backbone=best_backbone, learning_rate=best_learning_rate)
trainer = pl.Trainer(max_epochs=1)  # Change max_epochs to 1
trainer.fit(best_model, train_loader, test_loader)

# Evaluate on test set
trainer.test(best_model, test_loader)

[I 2024-04-28 19:36:22,663] A new study created in memory with name: no-name-f50e9157-6d51-4dff-ae11-9529e20877b9
/var/folders/5_/t39737k95_39n36vnz35wm100000gn/T/ipykernel_71485/3477474084.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /Users/maryamsoftdev/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████████████████████████████████| 20.4M/20.4M [00:12<00:00, 1.73MB/s]


Loaded pretrained weights for efficientnet-b0


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/maryamsoftdev/Downloads/lightning_logs

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | EfficientNet     | 4.0 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.061    Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/Users/maryamsoftdev/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/maryamsoftdev/anaconda3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


...

Testing: |                                                | 0/? [00:00<?, ?it/s]

python(73723) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(73728) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(73731) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(73734) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(73738) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(73741) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(73744) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(73748) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.2132299393415451
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


python(73754) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[{'test_loss': 0.2132299393415451}]